# Pre-canned Queries for Python
## Function to call Equity Reference Data
This query demonstrates how using a list of different identifier types and identifiers, the user can request:
  * Instrument level reference data including:
  * Swiss Stamp Tax Data
  * Instrument Identifiers
  * The name, and TRBC Scheme level 5 for the issuer of the instrument
  * Identify the Valuation Quote (the quote used to calculate market capitalization of the issue)
  * Quote level reference data for all active quotes of the instrument including:
  * The name of the exchange on which each quote trades
  * The RIC of the quote


tags :     `Equity`, `Reference`, `Swiss`, `Tax`, `Instrument`, `Identifiers`
## Import the library and load credentials
Credentials used by this notebook are stored in the config_files/credentials.ipynb. Please edit credentials.ipynb to set your credentials and run the next cell to continue with the scripts 

In [ ]:
%run ../config_files/credentials.ipynb 
%run ../config_files/environment_selection.ipynb

To connect to the Refinitiv Data and opening a session,given function is used. To find out more about the library,you can visit [refinitiv-data library](https://pypi.org/project/refinitiv-data/)

In [ ]:
session = rd.session.platform.Definition(
    app_key=APP_KEY,
    grant=rd.session.platform.GrantPassword(
        username=RD_LOGIN, password=RD_PASSWORD
    ),
).get_session()
session.open()


## Graph call
Graph call with given sets of samples below. 

In [ ]:
%%writefile query_cache
query Equity_ReferenceData($InstrumentList: [EDFInstrumentIdentifierArgs!]) {
  EDFInstrument(identifiers: $InstrumentList) {
    _metadata {
      identifier
    }
    ObjectId
    Reference {
      InstrumentCommonNames {
        InstrumentCommonName
        LanguageId
      }
      Cfi
      AssetClassId
      DtccEligible
      DerivedContent {
        SwissStampDutyTax {
          AssetTypeDescription
          IssuerDomicileText
          SwissStampDutyFlag
          TaxationComment
          TaxationType
        }
        UnitOfDelivery {
          UnitOfDeliveryCode
        }
      }
    }
    InstrumentIdentifiers(filter: {IdentifierData: {IdentifierValue: {EXISTS: true}}}) {
      IdentifierTypeId
      IdentifierType {
        IdentifierType {
          Name
        }
      }
      IdentifierData {
        IdentifierValue
      }
    }
    Issuer {
      ObjectId
      Names {
        Name {
          OrganizationName(filter: {NameTypeCode: {EQ: "LNG"}}) {
            Name
            LanguageId
          }
        }
      }
      ClassificationTrbcPrimarySchemaV2 {
        Industry {
          Name
        }
        Identifier {
          IndustryId
          Identifiers(filter: {ValueTypeId: {EQ: "320159"}}) {
            Value
            ValueTypeCode
            ValueTypeId
          }
        }
      }
    }
    ValuationQuote {
      ObjectId
    }
    AllQuotes(filter: {Reference: {QuoteIsActive: {EQ: true}}}) {
      ObjectId
      Reference {
        QuoteCurrencyId
        QuoteCountryId
        QuoteExchangeCode
      }
      MarketAttributableSource {
        Reference {
          Names(filter: {NameType: {EQ: "Long Name"}}) {
            LanguageId
            Name
            NameType
          }
        }
      }
      QuoteIdentifiers(filter: {OR: [{IdentifierTypeId: {EQ: 320110}}, {IdentifierTypeId: {EQ: 320048}}]}) {
        IdentifierTypeId
        IdentifierType {
          IdentifierType {
            Name
          }
        }
        IdentifierData {
          IdentifierEffectiveFrom
          IdentifierValue
        }
      }
    }
  }
}


## Sample :  Find by a mixture of Identifier Types

In [ ]:

variables = {'InstrumentList': [{'identifierTypeName': 'Isin', 'identifier': 'GB00B0SWJX34'}, {'identifierTypeName': 'ValorenNumber', 'identifier': '3886335'}, {'identifierTypeName': 'RIC', 'identifier': 'TSLA.O'}]}

In [3]:
output_format = 'handlebars'  # available: handlebars
handlebars_template = 'Equity Reference Data.html'
%run ../config_files/config_request_handler.ipynb

<table>
    <tbody>
    </tbody>
</table>

## Close the default session when done

In [ ]:
session.close()